In [68]:
import pandas as pd  
import numpy as np  

In [69]:
## Additional Functions needed

# Function to standardize marital status values
def standardize_cst_marital_status(x):
    x = x.strip().upper()  # Convert to uppercase for consistency
    
    if x == 'S':
        return 'Single'
    elif x == 'M':
        return 'Married'
    else:
        return 'Unknown'
    
# Function to standardize gender values
def standardization(x):
    if pd.isna(x):  # Check for NaN values
        return 'Unknown'
    
    x = x.upper()  # Convert to uppercase for consistency
    return 'Female' if x == 'F' else 'Male' if x == 'M' else 'Unknown'

In [70]:
import os
from datetime import datetime

In [71]:
def clean_cust_info(filepath: str) -> pd.DataFrame:
    """Cleans customer information data from a CSV file."""
    
    # Check if the file exists
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"CSV file not found: {filepath}")

    # Load the data
    df = pd.read_csv(filepath)

    # Check if the file is empty
    if df.empty:
        return "The file must have been empty"

    # Convert creation date to datetime
    df['cst_create_date'] = pd.to_datetime(df['cst_create_date'], format='%Y-%m-%d')

    # Rank by latest creation date
    df['latest_ranking'] = df.groupby('cst_id')['cst_create_date'].rank(method='first', ascending=False)

    # Filter only latest records
    new_df = df[df['latest_ranking'] == 1].copy()
    new_df.drop(columns=['latest_ranking'], inplace=True)  # Drop ranking column

    # Convert cst_id to integer 
    new_df['cst_id'] = new_df['cst_id'].astype(int)

    # Strip first and last names
    new_df['cst_firstname'] = new_df['cst_firstname'].str.strip()
    new_df['cst_lastname'] = new_df['cst_lastname'].str.strip()

    # Standardize gender and marital status
    new_df['cst_gndr'] = new_df['cst_gndr'].apply(standardization)
    new_df['cst_marital_status'] = new_df['cst_marital_status'].apply(standardize_cst_marital_status)
    new_df['dwh_create_date'] = datetime.now()
    return new_df

## Example usage:
filepath = "C:/Users/user/Desktop/FullDataAnalyticsProject/python_scripts/bronze/uncleaned_crm_datasets/crm_cust_info.csv"
cleaned_data = clean_cust_info(filepath=filepath)
print(cleaned_data.shape)

(18484, 8)


In [72]:
## Data Cleaning Checks
# Group by 'cst_id' and count occurrences, keeping NaN values
duplicates = cleaned_data.groupby('cst_id', dropna=False).size().reset_index(name='Number_of_duplicates')

# Filter for duplicates (count > 1) OR where 'cst_id' is NaN
duplicates_filtered = duplicates[duplicates['Number_of_duplicates'] > 1]

duplicates_filtered

,cst_id,Number_of_duplicates


In [73]:
cleaned_data[cleaned_data['cst_id'] == 29466]

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,dwh_create_date
18471,29466,AW00029466,Lance,Jimenez,Married,Male,2026-01-27,2025-03-27 17:04:26.539424


In [74]:
cleaned_data[(cleaned_data['cst_firstname'] != cleaned_data['cst_firstname'].str.strip())]

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,dwh_create_date


In [75]:
cleaned_data[(cleaned_data['cst_lastname'] != cleaned_data['cst_lastname'].str.strip())]

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,dwh_create_date


In [76]:
print(cleaned_data['cst_gndr'].unique())

['Male' 'Female' 'Unknown']


In [77]:
print(cleaned_data['cst_gndr'].value_counts())

cst_gndr
Male       7067
Female     6848
Unknown    4569
Name: count, dtype: int64


In [78]:
cleaned_data.head()

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,dwh_create_date
0,11000,AW00011000,Jon,Yang,Married,Male,2025-10-06,2025-03-27 17:04:26.539424
1,11001,AW00011001,Eugene,Huang,Single,Male,2025-10-06,2025-03-27 17:04:26.539424
2,11002,AW00011002,Ruben,Torres,Married,Male,2025-10-06,2025-03-27 17:04:26.539424
3,11003,AW00011003,Christy,Zhu,Single,Female,2025-10-06,2025-03-27 17:04:26.539424
4,11004,AW00011004,Elizabeth,Johnson,Single,Female,2025-10-06,2025-03-27 17:04:26.539424


In [79]:
cleaned_data[cleaned_data['cst_id'] == 29466]

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,dwh_create_date
18471,29466,AW00029466,Lance,Jimenez,Married,Male,2026-01-27,2025-03-27 17:04:26.539424


In [80]:
print(cleaned_data['cst_marital_status'].unique())

['Married' 'Single']


In [81]:
print(cleaned_data['cst_marital_status'].value_counts())

cst_marital_status
Married    10011
Single      8473
Name: count, dtype: int64


In [82]:
folder_path = "cleaned_datasets"
os.makedirs(folder_path, exist_ok=True)  # Avoids error if folder exists

print(f"Folder '{folder_path}' created successfully!")

Folder 'cleaned_datasets' created successfully!


In [83]:
cleaned_data.to_csv(f'{os.path.join('cleaned_datasets', 'crm_cust_info.csv')}', 
                    index=False, date_format="%y-%m-%d %H:%M:%S")